In [33]:

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from sklearn import tree as sktree

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [34]:
df_raw.head()

,File Name,File Version,Update Date,Region Code,State Code,Date Type,Date,Population age 18-64*,SSA Disability Beneficiaries age 18-64*,Percent of Adult Population Receiving SSA Adult Disability Benefits,Eligible Adult Population*,Adult Receipts,Eligible Adult Population Filing Rate,Favorable Adult Determinations,Eligible Adult Population Allowance Rate,All Adult Determinations,Adult Favorable Determination Rate,Population under age 18*,SSI Disabled Child (DC) Beneficiaries*,Percent of Population under age 18 Receiving SSI DC Benefits,Eligible Child Population*,SSI Disabled Child (DC) Receipts,Eligible Child Population Filing Rate,Favorable SSI Child (DC) Determinations,Eligible Child Population Allowance Rate,All SSI Disabled Child Determinations,SSI Disabled Child Allowance Rate,All Determinations,All Favorable Determinations,Favorable Determination Rate
0,SSA-SA-FYWL.csv,2,3/13/2023,SEA,AK,FY,2001,407208,12791,3.14,394417,3487,0.88,1573,0.40,3369,46.69,188453,970,0.51,187483.0,416,0.22,283,0.15,404,70.05,3773,1856,49.19
1,SSA-SA-FYWL.csv,2,3/13/2023,ATL,AL,FY,2001,2760170,211792,7.67,2548378,42416,1.66,15279,0.60,39021,39.16,1118057,25307,2.26,1092750.0,10602,0.97,4344,0.40,10879,39.93,49900,19623,39.32
2,SSA-SA-FYWL.csv,2,3/13/2023,DAL,AR,FY,2001,1636232,122081,7.46,1514151,24423,1.61,7931,0.52,24843,31.92,678719,13811,2.03,664908.0,6615,0.99,2281,0.34,6451,35.36,31294,10212,32.63
3,SSA-SA-FYWL.csv,2,3/13/2023,SFO,AZ,FY,2001,3209672,137268,4.28,3072404,31148,1.01,14446,0.47,27934,51.71,1408285,13416,0.95,1394869.0,4831,0.35,2599,0.19,4563,56.96,32497,17045,52.45
4,SSA-SA-FYWL.csv,2,3/13/2023,SFO,CA,FY,2001,21528766,890019,4.13,20638747,198004,0.96,83116,0.40,189867,43.78,9319305,85598,0.92,9233707.0,27018,0.29,14996,0.16,26280,57.06,216147,98112,45.39


In [35]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1092 entries, 0 to 1091
Data columns (total 30 columns):
 #   Column                                                               Non-Null Count  Dtype  
---  ------                                                               --------------  -----  
 0   File Name                                                            1092 non-null   object 
 1   File Version                                                         1092 non-null   int64  
 2   Update Date                                                          1092 non-null   object 
 3   Region Code                                                          1092 non-null   object 
 4   State Code                                                           1092 non-null   object 
 5   Date Type                                                            1092 non-null   object 
 6   Date                                                                 1092 non-null   int64  
 7   Popula

In [36]:
df_clean = df_raw.copy()

In [37]:
df_clean["SSA Disability Beneficiaries  age 18-64*"].head()

0     12791
1    211792
2    122081
3    137268
4    890019
Name: SSA Disability Beneficiaries  age 18-64*, dtype: object

In [38]:
df_clean["SSA Disability Beneficiaries  age 18-64*"] = (
    df_clean["SSA Disability Beneficiaries  age 18-64*"]
    .str.replace(",", "", regex=False)
    .astype(float)
)


In [39]:
df_clean["SSI Disabled Child (DC) Beneficiaries*"]
df_clean["SSI Disabled Child (DC) Beneficiaries*"] = (
    df_clean["SSI Disabled Child (DC) Beneficiaries*"]
    .str.replace(",", "", regex=False)
    .astype(float)
)

In [40]:
df_clean = df_clean[df_clean["Eligible Child Population*"].notna()]

In [41]:
df_clean[df_clean["SSI Disabled Child Allowance Rate"].isna()]


,File Name,File Version,Update Date,Region Code,State Code,Date Type,Date,Population age 18-64*,SSA Disability Beneficiaries age 18-64*,Percent of Adult Population Receiving SSA Adult Disability Benefits,Eligible Adult Population*,Adult Receipts,Eligible Adult Population Filing Rate,Favorable Adult Determinations,Eligible Adult Population Allowance Rate,All Adult Determinations,Adult Favorable Determination Rate,Population under age 18*,SSI Disabled Child (DC) Beneficiaries*,Percent of Population under age 18 Receiving SSI DC Benefits,Eligible Child Population*,SSI Disabled Child (DC) Receipts,Eligible Child Population Filing Rate,Favorable SSI Child (DC) Determinations,Eligible Child Population Allowance Rate,All SSI Disabled Child Determinations,SSI Disabled Child Allowance Rate,All Determinations,All Favorable Determinations,Favorable Determination Rate
819,SSA-SA-FYWL.csv,2,3/13/2023,NYC,PR,FY,2016,2069548,184949.0,8.94,1884599,11640,0.62,4775,0.25,11075,43.12,0,0.0,0.0,0.0,0,0.0,0,0.0,0,NaN,11075,4775,43.12
923,SSA-SA-FYWL.csv,2,3/13/2023,NYC,PR,FY,2018,1978423,168528.0,8.52,1809895,10772,0.60,2883,0.16,5913,48.76,0,0.0,0.0,0.0,0,0.0,0,0.0,0,NaN,5913,2883,48.76
975,SSA-SA-FYWL.csv,2,3/13/2023,NYC,PR,FY,2019,1941307,165207.0,8.51,1776100,8360,0.47,4969,0.28,10897,45.60,0,0.0,0.0,0.0,0,0.0,0,0.0,0,NaN,10897,4969,45.60
1027,SSA-SA-FYWL.csv,2,3/13/2023,NYC,PR,FY,2020,1916445,158314.0,8.26,1758131,9505,0.54,4331,0.25,9192,47.12,0,0.0,0.0,0.0,0,0.0,0,0.0,0,NaN,9192,4331,47.12
1079,SSA-SA-FYWL.csv,2,3/13/2023,NYC,PR,FY,2021,1916445,158314.0,8.26,1758131,9837,0.56,3975,0.23,9431,42.15,0,0.0,0.0,0.0,0,0.0,0,0.0,0,NaN,9431,3975,42.15


In [42]:
df_clean.loc[
    df_clean["SSI Disabled Child Allowance Rate"].isna(),
    ["All SSI Disabled Child Determinations", "Favorable SSI Child (DC) Determinations"]
]


,All SSI Disabled Child Determinations,Favorable SSI Child (DC) Determinations
819,0,0
923,0,0
975,0,0
1027,0,0
1079,0,0


In [43]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1091 entries, 0 to 1091
Data columns (total 30 columns):
 #   Column                                                               Non-Null Count  Dtype  
---  ------                                                               --------------  -----  
 0   File Name                                                            1091 non-null   object 
 1   File Version                                                         1091 non-null   int64  
 2   Update Date                                                          1091 non-null   object 
 3   Region Code                                                          1091 non-null   object 
 4   State Code                                                           1091 non-null   object 
 5   Date Type                                                            1091 non-null   object 
 6   Date                                                                 1091 non-null   int64  
 7   Population 

In [44]:
df_clean.to_csv("data/processed/ssa_clean.csv", index=False)

OSError: Cannot save file into a non-existent directory: 'data\processed'